## Data preparation for ear cascade training
Positive images are already in *AWEForSegmentation* dataset, we only need to make the description file with positions
of the masks. That is implemented in function `positive_description_file`.
The mask for photo `0417.png` was bigger than the photo, so I fixed that with the last function in the block below.

We still need to find negative images, that are images that don't include any ears.
For starters I got them with function `prepare_negative_from_positive`, that takes positive images and builds twice as
many negatives, by putting black and white squares over ears in the positive images. This function also writes the
negative description file for these images.

In [1]:
import cv2
import os

def positive_description_file():
    info = {}
    for photo in os.listdir('AWEForSegmentation/train'):
        mask = cv2.imread(f'./AWEForSegmentation/trainannot_rect/{photo}', 0).astype(bool)
        info[photo] = []
        unfinished = {}
        i = 0
        while i < mask.shape[0]:
            j = 0
            while j < mask.shape[1]:
                if mask[i, j]:
                    if i == 0 or not mask[i-1, j]:                  # upper row
                        if j == 0 or not mask[i, j-1]:              # upper left corner
                            unfinished[j] = {'start_y': i}          # there can only be one unfinished on y=j
                            width = 0
                            while j < mask.shape[1] and mask[i, j]:
                                j += 1
                                width += 1
                            unfinished[j-width]['width'] = width
                    elif mask[i-1, j] and mask[i+1, j]:             # left column
                        j += unfinished[j]['width']
                    elif mask[i-1, j] and not mask[i+1, j]:         # lower left corner
                        height = i - unfinished[j]['start_y'] + 1
                        info[photo].append([j, unfinished[j]['start_y'], unfinished[j]['width'], height])
                        j += unfinished[j]['width']
                    else:
                        print(photo, i, j)
                else:
                    j += 1
            i += 1
    with open('info.dat', 'w') as f:
        for photo in info:
            entry = f"AWEForSegmentation/train/{photo}  {len(info[photo])}  "
            for el in info[photo]:
                entry += ' '.join(map(str, el)) + "   "
            f.write(entry.strip() + "\n")
    return info

def prepare_negative_from_positive():
    info = []
    for photo in os.listdir('AWEForSegmentation/train'):
        mask = cv2.imread(f'./AWEForSegmentation/trainannot_rect/{photo}', 0).astype(bool)
        negative = cv2.imread(f'./AWEForSegmentation/train/{photo}', cv2.IMREAD_COLOR)
        negative[mask] = (0, 0, 0)
        cv2.imwrite(f'AWEForSegmentation/negative/n_{photo}', negative)
        info.append(f'AWEForSegmentation/negative/n_{photo}')
        negative[mask] = (255, 255, 255)
        cv2.imwrite(f'AWEForSegmentation/negative/n_1{photo[1:]}', negative)
        info.append(f'AWEForSegmentation/negative/n_1{photo[1:]}')
    with open('negative_start.txt', 'w') as f:
        for line in info:
            f.write(line + "\n")

def repair_mask_0417():
    """Mask of photo 0417.png is of size (360, 481) and photo is (360, 480)"""
    mask = cv2.imread(f'./AWEForSegmentation/trainannot_rect/0417.png', -1)
    new_mask = mask[:, :480]
    cv2.imwrite(f'AWEForSegmentation/trainannot_rect/0417.png', new_mask)

In [ ]:
repair_mask_0417()

In [ ]:
positive_description_file()
prepare_negative_from_positive()

### Different selections of negative images

#### Grayscale images
I downloaded a set of grayscale images from [Kaggle](https://www.kaggle.com/muhammadkhalid/negative-images) and saved
them to folder *neg_img_gs*.

In [2]:
def negative_description_file(path, file_name):
    """Make description file named 'file_name' for negative images in 'path'."""
    with open(file_name, 'w') as f:
        for photo in os.listdir(path):
            f.write(f'{path}/{photo}\n')

In [6]:
negative_description_file('neg_img_gs', 'negative_gs.txt')

#### Images from COCO
To find better negative images I used *COCO* dataset, to be able to select images with certain objects.
I downloaded  *2017 Train/Val annotations* from [COCO website](https://cocodataset.org/#download) and saved them to
folder *COCO/annotations*. From the annotations we only need *instances_train2017.json*, but it is too big to be
included on git, so you need to download it separately for this code to be reproducible.

Conda installation doesn't work on Windows, so I used `pip install pycocotools-windows` to get the tools needed to
download those images that contain the right objects.

In [3]:
from pycocotools.coco import COCO
coco = COCO('COCO/annotations/instances_train2017.json')

loading annotations into memory...
Done (t=29.09s)
creating index...
index created!


In [4]:
# display COCO categories
categories = coco.loadCats(coco.getCatIds())
names = [cat['name'] for cat in categories]
print('COCO categories: \n{}\n'.format(' .  '.join(names)))

COCO categories: 
person .  bicycle .  car .  motorcycle .  airplane .  bus .  train .  truck .  boat .  traffic light .  fire hydrant .  stop sign .  parking meter .  bench .  bird .  cat .  dog .  horse .  sheep .  cow .  elephant .  bear .  zebra .  giraffe .  backpack .  umbrella .  handbag .  tie .  suitcase .  frisbee .  skis .  snowboard .  sports ball .  kite .  baseball bat .  baseball glove .  skateboard .  surfboard .  tennis racket .  bottle .  wine glass .  cup .  fork .  knife .  spoon .  bowl .  banana .  apple .  sandwich .  orange .  broccoli .  carrot .  hot dog .  pizza .  donut .  cake .  chair .  couch .  potted plant .  bed .  dining table .  toilet .  tv .  laptop .  mouse .  remote .  keyboard .  cell phone .  microwave .  oven .  toaster .  sink .  refrigerator .  book .  clock .  vase .  scissors .  teddy bear .  hair drier .  toothbrush



In [5]:
ids = {}
# get all images containing given categories
ids["livingroom"] = coco.getImgIds(catIds=coco.getCatIds(catNms=['couch', 'chair', 'tv']))
ids["kitchen"] = coco.getImgIds(catIds=coco.getCatIds(catNms=['dining table', 'refrigerator', 'oven']))
ids["bedroom"] = coco.getImgIds(catIds=coco.getCatIds(catNms=['bed', 'book']))
ids["park"] = coco.getImgIds(catIds=coco.getCatIds(catNms=['bicycle', 'bench']))
ids["traffic"] = coco.getImgIds(catIds=coco.getCatIds(catNms=['bus', 'car', 'traffic light']))
ids["outside"] = coco.getImgIds(catIds=coco.getCatIds(catNms=['dog', 'backpack']))
ids["lunch"] = coco.getImgIds(catIds=coco.getCatIds(catNms=['table', 'cup', 'bowl', 'bottle']))
ids["office"] = coco.getImgIds(catIds=coco.getCatIds(catNms=['mouse', 'keyboard', 'laptop']))

people = coco.getImgIds(catIds=coco.getCatIds(catNms=['person']))

In [7]:
# separate photos that don't contain people, and those containing people that need to be manually checked for ears.
ids_no_people = []
ids_people = []
for key in ids:
    ids_no_people.extend([i for i in ids[key] if i not in people])
    ids_people.extend([i for i in ids[key] if i in people])

combined = list(dict.fromkeys(ids_no_people))
combined_with_people = list(dict.fromkeys(ids_people))

In [10]:
# download images
coco.download('no_people', combined)
coco.download('including_people', combined_with_people)

Cascade trained on photos without people (cascade in folder *cascadecoco*) still wasn't better from previous cascades,
it was still recognising many false positives. False positives were mainly on people, so I also downloaded photos that
include people, and then manually deleted those that include visible ears. In folder *coco_incl_people* I have
photos from folder *no_people* combined with photos from folder *including_people* that don't include ears
(cascade in *cascadecoco_people*).

For reproducing the code you don't need to manually delete images, but you should take the negative file directly from
my git repository - it includes a list of images that should be in the folder, so cascade will ignore the others.

In [9]:
negative_description_file('no_people', 'negative_coco.txt')

In [ ]:
# only run this, if you manually delete images that include ears - otherwise take 'negative_coco_people.txt' from git.
# negative_description_file('coco_incl_people', 'negative_coco_people.txt')

For final set of images I decided to combine all off the images gathered above, put them in folder *all_images*, and
create cascade in folder *cascade_combined*. To avoid having too many unneeded photos I took only photos with black
squares from *AWEForSegmentation/negative*.

In [11]:
negative_description_file('all_images', 'negative_all.txt')
